# **Idealista data cleaning**
*Jose Ramon Estevez Melgarejo 2022-03-10.*

## Introduction
This is the second step of my idealista portfolio´s project. 
In this notebook we will clean and structure the data we obtained with the web scraper that we created in the previous notebook called 'scraper.ipynb' located in the same repository. 

Data exploration has been done in advance so this notebook will just define the issues previously found and propose a way to clean or structure our data.

## 1. Importing Libraries & reading data

In [375]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import unicodedata

The first thing we are going to do is to import our csv file that contains the scrapped data into a Pandas DataFrame that we will call 'raw_data'.

In [376]:
raw_data = pd.read_csv('data/raw_data_22220305.csv').drop('Unnamed: 0', axis=1) # importing and removing 'Unnamed' empty column
raw_data.head()

,house_id,tittle,city,price_act,price_first,details_1_desc,details_2_desc,details_3_desc,advertiser
0,94283285,Flat / apartment for sale in Zona Bahía Blanca,Cadiz,"750,000","750,000","['262 m² built', '6 bedrooms', '4 bathrooms', ...","['Air conditioning', 'en trámite']",[],Professional advertiser
1,89113690,Duplex for sale in Mentidero - Teatro Falla - ...,Cadiz,"575,000","575,000","['135 m² built', '3 bedrooms', '3 bathrooms', ...",['en trámite'],[],Professional advertiser
2,95291210,Flat / apartment for sale in Mentidero - Teatr...,Cadiz,"320,000","320,000","['190 m² built', '8 bedrooms', '3 bathrooms', ...",['en trámite'],[],Professional advertiser
3,96746079,Flat / apartment for sale in Urb. alameda apod...,Cadiz,"390,000","390,000","['131 m² built, 117 m² floor area', '3 bedroom...","['Air conditioning', 'en trámite']",[],Professional advertiser
4,96622572,Flat / apartment for sale in Urb. Playa Santa ...,Cadiz,"720,000","720,000","['168 m² built, 167 m² floor area', '3 bedroom...",['en trámite'],[],Professional advertiser


Now we are going to have a look at the over all structure of our data. We will print the shape and data types of our data frame.

In [377]:
print(f'Input data has {raw_data.shape[0]} rows and {raw_data.shape[1]} columns')
raw_data.dtypes

Input data has 1218 rows and 9 columns


house_id           int64
tittle            object
city              object
price_act         object
price_first       object
details_1_desc    object
details_2_desc    object
details_3_desc    object
advertiser        object
dtype: object

From the output above we can see that our input data has 1218 rows and 9 columns. When we load our data from a csv, Pandas tries to infer the data types of each variable depending on the content so it is very likely that we will have to change some of our variables data types later on.

Next step is to check if there are any missing values. To do so I have created a function that will be handy specially when we try to extract information from detail columns. This function will return the number and percentage of missing values for each column in our data.

In [378]:
def na_detail(df):
    '''
    Recieves a data frame and returns a new data frame conatining theese 3 cols:
        - variable: variables of the imput data frame
        - na_count: sum of na 
        - na_per: % of na 
    '''
    result = pd.DataFrame(df.isna().sum(), columns = ['na_count']).reset_index(level=0)
    result.rename(columns={('index'): ('variable')}, inplace=True)
    result['na_per'] = round(result.na_count / df.shape[0] * 100, 2)

    return result


na_detail(raw_data)

,variable,na_count,na_per
0,house_id,0,0.00
1,tittle,0,0.00
2,city,0,0.00
3,price_act,0,0.00
4,price_first,0,0.00
5,details_1_desc,0,0.00
6,details_2_desc,0,0.00
7,details_3_desc,0,0.00
8,advertiser,6,0.49


We can see tha before doing any transformation to our data we only have 6 missing values in the advertiser column.

Now that we have a general view of our data structure we will go through each variable making the necesary changes. 

### house_id

*description: Unique property identifier (from idealista)*

There is not much to do with this variable as it has no other purpose than identifying the property. The only thing that we could do is to check for duplicated ids. 

In [379]:
# is there any duplicated house?
duplicated_ids = raw_data.house_id[raw_data.duplicated(subset=['house_id'])]
print(f'{len(duplicated_ids)} duplicated id/s')
if len(duplicated_ids) > 0:
    print(duplicated_ids.values)

1 duplicated id/s
[96404190]


apparently there is just one duplicated id (96404190) wich we can just proceed to delet.

In [380]:
# removing duplicated id
raw_data.drop_duplicates(subset='house_id', keep="last", inplace=True)
duplicated_ids = raw_data.house_id[raw_data.duplicated(subset=['house_id'])]
print(f'{len(duplicated_ids)} duplicated id/s')
if len(duplicated_ids) > 0:
    print(duplicated_ids.values)

0 duplicated id/s


In [381]:
raw_data.drop(771, axis=0, inplace=True) # 771 is the index position of the duplicate

### tittle
*description: tittle / short description of the property*

Having a look at this variable we see that there are two pieces of information we can obtain from it. 
The first one is the type of property (flat / apartment, duplex, house...) and the second one is about the area or zone where the property is located.Some times this information is the neighborhood and other times its the adress. Those two pieces of information seem to be separated by the phrase 'for sale in ' (the search was filtered on only properties for sale when scraping). We could then make two variables out of tittle, 'type' and 'zone'. 

To be sure that there is always the phrase 'for sale in ' we will do a small check:

In [382]:
raw_data.tittle = raw_data.tittle.str.lower() # converting to lowercase
raw_data.tittle.str.contains('for sale in ', na=False).sum() == raw_data.shape[0] # checking if for sale is in each row

True

Now that we have confirmed that 'for sale' is in each row, we can create 'type' and 'zone' columns

In [383]:
raw_data[['type', 'zone']] = raw_data.tittle.str.split('for sale in ',expand=True) # splitting tittle into type and zone
raw_data[['tittle', 'type', 'zone']].head()

,tittle,type,zone
0,flat / apartment for sale in zona bahía blanca,flat / apartment,zona bahía blanca
1,duplex for sale in mentidero - teatro falla - ...,duplex,mentidero - teatro falla - alameda
2,flat / apartment for sale in mentidero - teatr...,flat / apartment,mentidero - teatro falla - alameda
3,flat / apartment for sale in urb. alameda apod...,flat / apartment,"urb. alameda apodaca, mentidero - teatro falla..."
4,flat / apartment for sale in urb. playa santa ...,flat / apartment,"urb. playa santa maría del mar, playa stª mª d..."


### city
*description: city where the property is located plus some zone info in some cases*

This column is a bit triky. We can find in this column two options:
- Just the city name
- Neigborhood plus city name

When there is information about the neigborhood, we observe that our 'zone' column that we have just created contains the exact adress or at least more detailed information about the location of the property (calle means street in spanish):

In [384]:
raw_data.query("city != 'Cadiz'")[['city', 'zone']].head()

,city,zone
8,"La Caleta - La Viña, Cadiz",calle obispo urquinaona
13,"Centro Histórico - Plaza España, Cadiz",paseo de la alameda apodaca nn
14,"Mentidero - Teatro Falla - Alameda, Cadiz",calle navas
33,"Playa Victoria, Cadiz",av. andalucía
40,"La Paz - Segunda Aguada - Loreto, Cadiz",calle carraca


If there is only the city name, we observe that our 'zone' column contains the neigborhood information:

In [385]:
raw_data.query("city == 'Cadiz'")[['city', 'zone']].head()

,city,zone
0,Cadiz,zona bahía blanca
1,Cadiz,mentidero - teatro falla - alameda
2,Cadiz,mentidero - teatro falla - alameda
3,Cadiz,"urb. alameda apodaca, mentidero - teatro falla..."
4,Cadiz,"urb. playa santa maría del mar, playa stª mª d..."


We see that neighborhoos and adress information is mixed among both columns, we need to find a way to separate them.

The reason why I think this is hapening is because idealista.com gives you the chance to hide your adress but you have to pay. If you pay, only information about the neigborhood will be posted (in the tittle section) and if you dont the location will be exposed.

So we know that we will get the neigborhood for all properties but we will not know the adress of all properties. On the other hand we will obtain information about how many people did pay for this service even if this information is irelevant.

Our goal is to create three columns, nbhood (all properties), street (if there is), st_number (if there is).

The strategy will be:
- If only the city is available in city column: nbhood will be extracted from 'zone' column. 'street' and 'st_number' will be empty.
- If neigborhood is available in city column: nbhood will be extracted from 'city' column. 'street' and 'st_number' will be extracted from 'zone' column.

Aditionaly we will do some string cleaning and standarisation on our nbhood and street columns

In [386]:
raw_data['nbhood'] = [l.replace(', Cadiz', '').replace('Cadiz', '') for l in raw_data.city] # creating a col with nbhood from city col
raw_data['nbhood'] = np.where(raw_data['nbhood'] == '', raw_data['zone'], raw_data['nbhood']) # adding  nbhood from zone if not in city col


raw_data['street'] = np.where(raw_data['city'] != 'Cadiz', raw_data['zone'], 'unknown') # creating a col with street from zone col
raw_data[['street', 'st_number']] = raw_data.street.str.split(pat=',',expand=True) # separating st_number from street
raw_data['st_number'] = raw_data['st_number'].str.strip() # removing empty spaces
raw_data['st_number'] = raw_data.st_number.str.split(pat=' ',expand=True)[0] # selecting only the first number (5 a, 11 duplicado, etc...)
raw_data['st_number'] = pd.to_numeric(raw_data['st_number'], errors='coerce') # converting to numeric

# removing accents and transforming to lowe case nbhood and street
def str_stand(s):
    'removes accents and converts str to lower case'
    result = ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')
    result = result.lower()
   
    return result

raw_data['nbhood'] = list(map(str_stand, raw_data['nbhood']))
raw_data['street'] = list(map(str_stand, raw_data['street']))

# Replacing manually the name of some neighborhoods
custom_equivalences_dict = {
    'urb. playa santa maria del mar, playa stª mª del mar':'playa stª mª del mar',
    'urb. alameda apodaca, mentidero - teatro falla - alameda':'mentidero - teatro falla - alameda',
    'urb. loreto, la paz - segunda aguada - loreto':'la paz - segunda aguada - loreto',
    'urb. santa maria del mar, playa stª mª del mar':'playa stª mª del mar',
    'urb. comisaria, zona san jose - varela':'zona san jose - varela',
    'urb. san jose-varela-la laguna, zona laguna':'zona laguna',
}
raw_data.replace({"nbhood": custom_equivalences_dict}, inplace=True)

raw_data[['city', 'nbhood', 'street', 'st_number']].head()

,city,nbhood,street,st_number
0,Cadiz,zona bahia blanca,unknown,NaN
1,Cadiz,mentidero - teatro falla - alameda,unknown,NaN
2,Cadiz,mentidero - teatro falla - alameda,unknown,NaN
3,Cadiz,mentidero - teatro falla - alameda,unknown,NaN
4,Cadiz,playa stª mª del mar,unknown,NaN


In [387]:
raw_data.drop(['city', 'zone'], axis=1, inplace=True) # removing this cols, city is always Cadiz and zone is already captured by nbhood, street and st_number

### price_act &	price_first	
*description: actual price and first published price in Euros*

We saw previously when we printed all variables data types that theese two variables where cataloged as 'object' meanning that they are not a number. There mus be some king of non numeric character in these variables. Lets append both columns and have a look at what type of values we have:

In [388]:
raw_data['price_act'].append(raw_data['price_first']).value_counts()

180,000          69
330,000          62
165,000          58
200,000          52
250,000          48
                 ..
\n139,000 €\n     1
\n420,000 €\n     1
\n350,000 €\n     1
\n135,000 €\n     1
\n123,000 €\n     1
Length: 341, dtype: int64

Just from printing the value count we can see that there is a thousand separator as well as some other characters such as €\n.
We will now define a function that will clean the variable from those non numeric characters and convert to float64.

In [389]:
def clean_num(array):
    '''cleans number & transforms column to float64'''
    if array.dtypes != 'float64':
        result = []
        for elemen in array:
            number = [l for l in elemen if l.isdigit()]
            result.append(''.join(number))
        
        result = [float(l) for l in result]
        return result
    
    else:
        return array

raw_data.price_act = clean_num(raw_data.price_act)
raw_data.price_first = clean_num(raw_data.price_first)

raw_data[['price_act', 'price_first']].dtypes 

price_act      float64
price_first    float64
dtype: object

### details_x_desc
*description: lists of properties characteristics*

These three columns are the most complex ones. Each row contains a list of characteristics. There are some compulsory characteristics such as number of bedrooms or m2 buit. There are also some characteristics not compulsory like air conditioning. We need to make an asumption here, if there is no mentoin about balcony for example we will asume that the property has no balcony. Some of theese fields are not mandatory when uploading a property to idealista but we will still asume that if the characteristic is not listed it just doesn´t exist.

In [390]:
raw_data[['details_1_desc', 'details_2_desc', 'details_3_desc']].head()

,details_1_desc,details_2_desc,details_3_desc
0,"['262 m² built', '6 bedrooms', '4 bathrooms', ...","['Air conditioning', 'en trámite']",[]
1,"['135 m² built', '3 bedrooms', '3 bathrooms', ...",['en trámite'],[]
2,"['190 m² built', '8 bedrooms', '3 bathrooms', ...",['en trámite'],[]
3,"['131 m² built, 117 m² floor area', '3 bedroom...","['Air conditioning', 'en trámite']",[]
4,"['168 m² built, 167 m² floor area', '3 bedroom...",['en trámite'],[]


If we select the first element from 'details_1_desc' we can see that the list is actually readed as a string. So the first thing we have to do is to convert the string to an actual list. Then we will apend the lists of the three 'datil_x_desc' columns into one sole column so that we do the work just once.

In [391]:
raw_data.details_1_desc[0]

"['262 m² built', '6 bedrooms', '4 bathrooms', 'Terrace', 'Balcony', 'Additional parking space for € 35,000', 'Second hand/good condition', 'Fitted wardrobes', 'Orientation South, East', 'Built in 1973', 'Individual heating', '10th floor exterior', 'with lift']"

In [392]:
# converting details_123_desc to a list and combining all into only one list col
raw_data.details_1_desc = raw_data.details_1_desc.map(lambda x: x.strip("[]").replace("'", "").lower().split(", "))
raw_data.details_2_desc = raw_data.details_2_desc.map(lambda x: x.strip("[]").replace("'", "").lower().split(", "))
raw_data.details_3_desc = raw_data.details_3_desc.map(lambda x: x.strip("[]").replace("'", "").lower().split(", "))
raw_data['details'] = raw_data['details_1_desc'] + raw_data['details_2_desc'] + raw_data['details_3_desc']
raw_data['details'] = raw_data['details'].apply(lambda x: list(set(x))) # removing posible duplicates
raw_data['details'] = raw_data['details'].apply(lambda x: [l for l in x if l != ""]) # removing empty details from the lists
raw_data.drop(['details_1_desc', 'details_2_desc', 'details_3_desc'], axis=1, inplace=True) # dropping old cols
raw_data.head()

,house_id,tittle,price_act,price_first,advertiser,type,nbhood,street,st_number,details
0,94283285,flat / apartment for sale in zona bahía blanca,750000.0,750000.0,Professional advertiser,flat / apartment,zona bahia blanca,unknown,NaN,"[built in 1973, orientation south, with lift, ..."
1,89113690,duplex for sale in mentidero - teatro falla - ...,575000.0,575000.0,Professional advertiser,duplex,mentidero - teatro falla - alameda,unknown,NaN,"[storeroom, 2nd floor exterior, individual hea..."
2,95291210,flat / apartment for sale in mentidero - teatr...,320000.0,320000.0,Professional advertiser,flat / apartment,mentidero - teatro falla - alameda,unknown,NaN,"[8 bedrooms, west, 190 m² built, built in 1888..."
3,96746079,flat / apartment for sale in urb. alameda apod...,390000.0,390000.0,Professional advertiser,flat / apartment,mentidero - teatro falla - alameda,unknown,NaN,"[storeroom, orientation south, with lift, buil..."
4,96622572,flat / apartment for sale in urb. playa santa ...,720000.0,720000.0,Professional advertiser,flat / apartment,playa stª mª del mar,unknown,NaN,"[with lift, 167 m² floor area, orientation sou..."


We can now check that the first element of details column is actually casted as a list

In [393]:
raw_data.details[0]

['built in 1973',
 'orientation south',
 'with lift',
 'terrace',
 '262 m² built',
 'east',
 'individual heating',
 'fitted wardrobes',
 '10th floor exterior',
 'air conditioning',
 'additional parking space for € 35,000',
 'second hand/good condition',
 '4 bathrooms',
 'balcony',
 '6 bedrooms',
 'en trámite']

It would be a good idea to see what type of details we have and how often they occur in our data set. To do so I have crated a function that will remove any number from details list and count the number of ocurences. The Data Frame returned will be very usefull to give us an idea of what information we can extract from these lists.

In [394]:
def details_desc(series):
    '''
    Takes a df.column composed by lists, removes any number from the lists and returns a df 
    with the count of ocurrence of each element and percentage of ocurrence in the df.column.
    '''
    result = []
    for list in series:
        list = [l.replace(',', '').strip() for l in list] 
        list = [''.join(re.findall(r'\D', l)) for l in list] # removing all numbers
        for element in list:
            result.append(element)
    
    result = Counter(result)
    result = pd.DataFrame(result.items(), columns=['characteristic', 'count'])
    result['per'] = round(result['count'] / len(series) * 100)
    result.sort_values('count', inplace=True, ascending = False)
    result = result.query("characteristic != ''")
    return(result)
        

all_details = details_desc(raw_data.details)
all_details.head(10)

,characteristic,count,per
4,m² built,1216,100.0
11,second hand/good condition,1098,90.0
14,bedrooms,1058,87.0
26,m² floor area,735,60.0
35,bathroom,692,57.0
2,with lift,672,55.0
7,fitted wardrobes,649,53.0
15,en trámite,628,52.0
0,built in,562,46.0
12,bathrooms,521,43.0


We can see for example that all properties have information about m2 built but not all have information about the year they where built.

Now we have to find a way to extract information from these lists. To do so I have created two different functions:

**extract_num :** It will loop through each element of the list and filter those that contain the keyword then it will return the number asociated to the key word. Example: using the key_word: 'bedroom' in this list [3 bedrooms, 1 balcony, with lift] will return 3.

**extract_bol :** It will loop through each element of the list and filter those that contain the keyword. If any element matches the key_word 1 will be returned if not 0. Example: using the key_word: 'fitted wardrobes' in this list [3 bedrooms, 1 balcony, with lift] will return 0.


In [395]:
def extract_num(series, key_word):
    '''
    Takes a df.column composed by lists and a keyword.
    Filters each list of the series depending on if it contains the keyword or not.
        - If no element is found containing the keyword, np.nan will be returned.
        - If there is a number in the filtered element it returns the number.
        - If the keyword is found but no number is asociated it will return 0.
    The output is a list of all numbers asociated.
    '''
    result = []
    for list in series:
        detail = [l.lower().replace(',', '') for l in list if key_word.lower() in l] # finds elements that contain the key_word
        if len(detail) > 0:
            num = re.findall(r'\d+', detail[0])
            
            if len(num) > 0:
                result.append(float(num[0]))
            else: 
                result.append(0)
        else:
            result.append(np.nan)
            
    return(result)

def extract_bol(series, key_word):
    '''   
    Takes a df.column composed by lists and a keyword.
    Filters each list of the series depending on if it contains the keyword or not.
        - If more than 0 element are found containing the keyword 1 will be returned.
        - If no element are found containing the keyword 0 will be returned.
    '''
    result = []
    for list in series:
        detail = [l.lower() for l in list if key_word.lower() in l]
        if len(detail) > 0:
            result.append(1)
        else: 
            result.append(0)
            
    return(result)

This is just an auxiliary funtion to obtain row totals.

In [396]:
def adorn_totals(df):
    '''
    Takes a pd.DataFrame and returns the same df but with a totals row
    '''
    totals = df.sum()
    mask = [type(l)==str for l in totals]
    totals[mask] = 'Total'
    result = df.append(totals, ignore_index=True)
    return result

### sq_m
*description: square meters built*

Lets try our function using the keyword 'm² built'. We saw that 100% of the properties had this information:

In [397]:
all_details.query("characteristic.str.contains('m² built')")

,characteristic,count,per
4,m² built,1216,100.0


If we use the keyword 'm² built' we should get a number for each property. Lets extract the number, create a new column called 'sq_m' and check for na:

In [398]:
raw_data['sq_m'] = extract_num(raw_data.details, 'm² built')
na_detail(raw_data).query("variable.str.contains('sq_m')")

,variable,na_count,na_per
10,sq_m,0,0.0


We see that no na where generated. Everything seems to be working properly. Nevertheless we will print some general information to be more confident about our results

In [399]:
raw_data.sq_m.describe()

count    1216.000000
mean      110.807566
std        86.840846
min        27.000000
25%        70.000000
50%        92.000000
75%       134.000000
max      2200.000000
Name: sq_m, dtype: float64

### bedrooms & bathrooms
*description: number of bedrooms or bathrooms*

We will repeat the same process for number of bedrooms and bathrooms. We see that if we apply the keyword 'bedroom' a number will be returned for 100% of the properties. 

In [400]:
all_details.query("characteristic.str.contains('bedroom')").pipe(adorn_totals)

,characteristic,count,per
0,bedrooms,1058,87.0
1,bedroom,138,11.0
2,no bedroom,19,2.0
3,or more bedrooms,1,0.0
4,Total,1216,100.0


In [401]:
raw_data['bedrooms'] = extract_num(raw_data.details, 'bedroom')
na_detail(raw_data).query("variable.str.contains('bedrooms')")

,variable,na_count,na_per
11,bedrooms,0,0.0


Same hapends with keyword bathroom:

In [402]:
all_details.query("characteristic.str.contains('bathroom')").pipe(adorn_totals)

,characteristic,count,per
0,bathroom,692,57.0
1,bathrooms,521,43.0
2,no bathroom,3,0.0
3,Total,1216,100.0


In [403]:
raw_data['bathrooms'] = extract_num(raw_data.details, 'bathroom')
na_detail(raw_data).query("variable.str.contains('bathrooms')")

,variable,na_count,na_per
12,bathrooms,0,0.0


### const_year
*description: year of construction*

From our all_details data frame we can see that only 46% of the properties have information about the construction year. We will still try to extract it but i am not sure if it will be of any use in the future.

In [404]:
all_details.query("characteristic.str.contains('built in')").pipe(adorn_totals)

,characteristic,count,per
0,built in,562,46.0
1,Total,562,46.0


In [405]:
raw_data['const_year'] = extract_num(raw_data.details, 'built in')
na_detail(raw_data).query("variable.str.contains('const_year')")

,variable,na_count,na_per
13,const_year,654,53.78


### floor
*description: floor of the property*

If we use the key_word 'floor' to filter the characteristics we see that there are some characteristics that are not related to the floor number. These are 'm² floor area' and 'floors'. We should replace those descriptions so we can extract the floor number with no interference. Same thing happends with 'floors', it looks like it refers to the number of floors the property has more than the actual floor number of the house so we can replace it for 'decks' for example. 

In [406]:
all_details.query("characteristic.str.contains('floor')").pipe(adorn_totals)

,characteristic,count,per
0,m² floor area,735,60.0
1,st floor exterior,208,17.0
2,rd floor exterior,192,16.0
3,th floor exterior,181,15.0
4,nd floor exterior,159,13.0
5,ground floor exterior,130,11.0
6,rd floor,38,3.0
7,ground floor,38,3.0
8,ground floor interior,38,3.0
9,nd floor,32,3.0


So we will firs replace those descriptions from the details lists and then extract the floor number. To replace descriptions in the lists i have created the following function:

In [407]:
def replace_str(array, str, repl_str):
    '''
     Takes:
        - array: df.column composed by lists
        - str: string to be replaced
        - repl_str: string to replace
    Returns the same array with the string replaced.
    '''
    result = []
    for list in array:
        new_list = [l.replace(str, repl_str) for l in list]
        result.append(new_list)
    return result

In [408]:
raw_data.details = replace_str(raw_data.details, 'm² floor area', 'sq_m_cons')
raw_data.details = replace_str(raw_data.details, 'floors', 'decks')

If we create again 'all_details' df and check the keyword 'floor' we can see that all characteristic listed are now referring to the floor number and we are ready to extract the number.

In [409]:
all_details = details_desc(raw_data.details)
all_details.query("characteristic.str.contains('floor')").pipe(adorn_totals)

,characteristic,count,per
0,st floor exterior,208,17.0
1,rd floor exterior,192,16.0
2,th floor exterior,181,15.0
3,nd floor exterior,159,13.0
4,ground floor exterior,130,11.0
5,rd floor,38,3.0
6,ground floor,38,3.0
7,ground floor interior,38,3.0
8,nd floor,32,3.0
9,st floor,32,3.0


In [410]:
raw_data['floor'] = extract_num(raw_data.details, 'floor')
na_detail(raw_data).query("variable.str.contains('floor')")

,variable,na_count,na_per
14,floor,72,5.92


From  'na_detail' we confirm that we extracted the floor number of aprox 94% of our data 

### lift
*description: if the proferty has lift or not*
97% of the properties have information about if it has lift or not. We will just use our 'extract_bol' to check if the key_word 'with lift' is there or not. We will consider that the 3% of properties with no info about lift do not have it.

In [411]:
all_details.query("characteristic.str.contains('lift')").pipe(adorn_totals)

,characteristic,count,per
0,with lift,672,55.0
1,no lift,509,42.0
2,Total,1181,97.0


In [412]:
raw_data['lift'] = extract_bol(raw_data.details, 'with lift')
na_detail(raw_data).query("variable.str.contains('lift')")

,variable,na_count,na_per
15,lift,0,0.0


### balcony
*description: if the proferty has balcony or not*

In [413]:
raw_data['balcony'] = extract_bol(raw_data.details, 'balcony')

### air_con
*description: if the proferty has air_con or not*

In [414]:
raw_data['air_con'] = extract_bol(raw_data.details, 'air conditioning')

### ft_wardrobes
*description: if the proferty has ft_wardrobes or not*

In [415]:
raw_data['ft_wardrobes'] = extract_bol(raw_data.details, 'fitted wardrobes')

### terrace
*description: if the proferty has terrace or not*

In [416]:
raw_data['terrace'] = extract_bol(raw_data.details, 'terrace')

### exterior
*description: if the proferty has exterior views or not*

In [417]:
all_details.query("characteristic.str.contains('ext')").pipe(adorn_totals)

,characteristic,count,per
0,st floor exterior,208,17.0
1,rd floor exterior,192,16.0
2,th floor exterior,181,15.0
3,nd floor exterior,159,13.0
4,ground floor exterior,130,11.0
5,mezzanine exterior,18,1.0
6,exterior,10,1.0
7,Total,898,74.0


In [418]:
raw_data['exterior'] = extract_bol(raw_data.details, 'exterior')

### heating
*description: type of heating system*

As we can see from our all_details data frame, there are many categories for heating system. In order to simplify this column we will only account for three types: no heating, individual heating and central heating. If no information is found about heating we will consider that there is no heating system.

In [419]:
all_details.query("characteristic.str.contains('heating')").pipe(adorn_totals)

,characteristic,count,per
0,no heating,518,43.0
1,individual heating,58,5.0
2,central heating,28,2.0
3,individual heating: electric,22,2.0
4,individual heating: air conditioning/heat pump,18,1.0
5,individual heating: natural gas,14,1.0
6,central heating: gas,6,0.0
7,individual heating: propane/butane gas,2,0.0
8,Total,666,54.0


We then have to make some replacements in the lists contained in our detail columns

In [420]:
raw_data.details = replace_str(raw_data.details, 'individual heating: electric', 'individual heating')
raw_data.details = replace_str(raw_data.details, 'individual heating: air conditioning/heat pump', 'individual heating')
raw_data.details = replace_str(raw_data.details, 'individual heating: natural gas', 'individual heating')
raw_data.details = replace_str(raw_data.details, 'individual heating: propane/butane gas', 'individual heating')
raw_data.details = replace_str(raw_data.details, 'central heating: gas', 'central heating')

all_details = details_desc(raw_data.details)
all_details.query("characteristic.str.contains('heating')").pipe(adorn_totals)

,characteristic,count,per
0,no heating,518,43.0
1,individual heating,114,9.0
2,central heating,34,3.0
3,Total,666,55.0


Now we can extract heating info into one column 'heating'.

In [421]:
# Creating one separate column for each type of heating plus one unknown col if no type of heating is extracted
raw_data['individual'] = extract_bol(raw_data.details, 'individual heating') 
raw_data['central'] = extract_bol(raw_data.details, 'central heating')
raw_data['no'] = extract_bol(raw_data.details, 'no heating')
raw_data['unknown'] = np.where(raw_data['individual'] + raw_data['central'] + raw_data['no'] == 0, 1, 0) 

print((raw_data['individual'] + raw_data['central'] + raw_data['no'] + raw_data['unknown']).unique() == 1) # checking that every row is in one of 4 categories
raw_data['heating'] = raw_data[['individual', 'central', 'no', 'unknown']].idxmax(1) # combining all cols into one
raw_data.heating.replace('unknown', 'no', inplace=True) # replacing 'unknown' for no. no info no heating.
raw_data.drop(['individual', 'central', 'no', 'unknown'], axis=1, inplace=True) # droping individual cols

[ True]


In [422]:
na_detail(raw_data).query("variable.str.contains('heating')")

,variable,na_count,na_per
21,heating,0,0.0


Once there is no more information to extract from 'details' column we can proceed and drop it.

In [423]:
raw_data.drop(['details', 'tittle'], axis=1, inplace=True)

### Writting final data into a .csv file

In [424]:
raw_data.to_csv('data/clean_data_3.csv', index=False)